In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
from haversine import haversine
from shapely.geometry import LineString
from shapely.ops import split, linemerge
import krippendorff

In [2]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [4]:
#split line into segments of 30 ft
def split_line_at_intervals(line, interval=30):
    distance = 0
    segments = []

    while distance < line.length:
        sub_line = line.interpolate(distance)
        next_distance = distance + interval
        next_sub_line = line.interpolate(next_distance)
        segment = LineString([sub_line, next_sub_line])
        segments.append(segment)
        distance = next_distance

    return segments

def split_geodataframe(geodf, local_crs, interval=30):
    
    geodf = geodf.to_crs(local_crs)
    
    new_rows = []

    for index, row in geodf.iterrows():
        line = row.geometry
        splitted_lines = split_line_at_intervals(line, interval)

        for segment_id, segment in enumerate(splitted_lines):
            new_row = row.copy()
            new_row.geometry = segment
            new_row['segment_id'] = segment_id
            new_rows.append(new_row)

    df = gpd.GeoDataFrame(new_rows, columns=geodf.columns.append(pd.Index(['segment_id'])))
    #add a column that combines street_edge_id and segment_id
    df['street_edge_segment_id'] =  df['street_edge_id'].astype(str) + '_' +  df['segment_id'].astype(str)
    df.crs = local_crs
    return df

In [5]:
#read in seattle validation labels
seattle_labels = gpd.read_file('data/round-1/seattle-validation-labels-round1-230611.geojson')
seattle_labels = seattle_labels.drop(columns=['time_created'])
seattle_labels.head()

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,3,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,t,3.0,f,pole,None,POINT (-122.32071 47.66104)
1,4,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,t,2.0,f,pole,None,POINT (-122.32070 47.66103)
2,5,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,t,2.0,f,bumpy,None,POINT (-122.32071 47.66106)
3,8,Devon Snyder,8,480,University District,Occlusion,47.661106,-122.320724,OH4yKYW98UY1aMK_b68Etw,None,NaN,f,None,None,POINT (-122.32072 47.66111)
4,9,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,t,3.0,f,sign,temporary due to construction,POINT (-122.32066 47.66110)


In [6]:
#selec label type == Crosswalk
seattle_labels[seattle_labels['label_type'] == 'Crosswalk']



,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
6,14,Devon Snyder,14,482,University District,Crosswalk,47.661316,-122.320869,O0VZ-MF6Nrmjhkb2nCRhCw,None,2.0,f,None,None,POINT (-122.32087 47.66132)
7,15,Devon Snyder,15,480,University District,Crosswalk,47.661205,-122.320778,O0VZ-MF6Nrmjhkb2nCRhCw,t,2.0,f,paint fading,None,POINT (-122.32078 47.66121)
16,28,Devon Snyder,28,483,University District,Crosswalk,47.661461,-122.320740,qCkbp2lszdGrR5s3hdSiTQ,t,3.0,f,paint fading,None,POINT (-122.32074 47.66146)
76,198,Devon Snyder,198,179,International District,Crosswalk,47.600964,-122.334213,1mcZ_CqLAp3r4RPkBcqWYg,t,2.0,f,None,None,POINT (-122.33421 47.60096)
78,200,Devon Snyder,200,208,International District,Crosswalk,47.600895,-122.334084,BsqrOdDSbJWA6WB0NVJeiA,t,2.0,f,None,None,POINT (-122.33408 47.60089)
80,202,Devon Snyder,202,203,International District,Crosswalk,47.600807,-122.334206,BsqrOdDSbJWA6WB0NVJeiA,t,2.0,f,None,None,POINT (-122.33421 47.60081)
88,212,Devon Snyder,212,190,International District,Crosswalk,47.600883,-122.334366,1mcZ_CqLAp3r4RPkBcqWYg,t,2.0,f,None,None,POINT (-122.33437 47.60088)
103,227,Devon Snyder,227,243,International District,Crosswalk,47.600891,-122.335480,f4p1g4gBWGcBH_vl47_A8Q,t,2.0,f,None,None,POINT (-122.33548 47.60089)
115,239,Devon Snyder,239,184,International District,Crosswalk,47.600048,-122.335510,ZMVFCSQHaz1q8y3LB_IyOw,None,1.0,f,None,None,POINT (-122.33551 47.60005)
123,251,Devon Snyder,251,249,International District,Crosswalk,47.600418,-122.335999,4Q1kZf3mWO6o4XKS-k88Vw,t,1.0,f,None,None,POINT (-122.33600 47.60042)


In [8]:
#read in seattle streets
seattle_streets = gpd.read_file("data/round-1/seattle-streets-230611.geojson")

In [9]:
#obtain cluster to street segment mapping
def cluster_to_street_segment_mapping(df,streets_seg):
     # Group by cluster_id and combine all geometries in each group into a MultiPoint
     df_grouped = df.groupby('cluster_id')['geometry'].apply(lambda x: x.unary_union)
     # Calculate centroid of each group
     centroids = df_grouped.centroid
     # Convert centroids to GeoDataFrame
     cluster_centroids = gpd.GeoDataFrame(geometry=centroids).reset_index()
     cluster_centroids.crs = {'init' :'epsg:4326'}
     cluster_centroids_ft = cluster_centroids.to_crs(epsg=streets_seg.crs.to_epsg())
     #map gdf_centroids to seattle_split, delete index_right column
     cluster_centeroids_street = cluster_centroids_ft.sjoin_nearest(streets_seg, distance_col='distance')
     cluster_centeroids_street = cluster_centeroids_street.drop(columns=['index_right'])
     #select only cluster_id and street_edge_segment_id
     cluster_centeroids_street = cluster_centeroids_street[['cluster_id','street_edge_segment_id']]
     #if there are multiple street_edge_segment_ids for a cluster_id, select the first one
     cluster_centeroids_street = cluster_centeroids_street.groupby('cluster_id').first().reset_index()
     return cluster_centeroids_street

In [10]:
#construct agreement table
def irr_table (df,street_seg):
    mikey = df[df['username'] == 'mikey']
    devon = df[df['username'] == 'Devon Snyder']
    irr_mikey = mikey.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    irr_devon = devon.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    #replace usernames with 1
    irr_mikey  = irr_mikey .replace('mikey',1)
    irr_devon  = irr_devon .replace('Devon Snyder',1)
    #group by street_edge_segment_id and count the number of label_ids
    irr_mikey = irr_mikey.groupby('street_edge_segment_id').sum()
    irr_devon = irr_devon.groupby('street_edge_segment_id').sum()
    irr_mikey = irr_mikey[['username']].reset_index()
    irr_devon = irr_devon[['username']].reset_index()
    irr_mikey = irr_mikey.rename(columns={'username':'mikey'})
    irr_devon = irr_devon.rename(columns={'username':'devon'})
    irr_mikey_devon = irr_mikey.merge(irr_devon, on='street_edge_segment_id', how='left').fillna(0)
    irr_mikey_devon = irr_mikey_devon.drop(columns=['street_edge_segment_id'])
    
    return irr_mikey_devon

In [14]:
def krippendorff_alpha(labels,label_type_id, streets, local_crs):
    #clustering by label type
    label_type = cluster_label_type_at_index(labels,label_type_id)[2]
    #split streets into segments
    street_seg = split_geodataframe(streets, local_crs, interval=30)
    #map cluster_id to street_edge_segment_id
    mapping = cluster_to_street_segment_mapping(label_type,street_seg)
    #update the street_edge_segment_id with the one closest to the cluster centroid
    label_type = label_type.merge(mapping, on='cluster_id', how='left')
    #select only the columns we need
    df = label_type[['username','cluster_id','street_edge_segment_id','label_id']]
    #construct agreement table
    irr = irr_table(df,street_seg)
    data_matrix = irr.T.to_numpy()
    alpha = krippendorff.alpha(data_matrix,level_of_measurement='ratio')

    return alpha.round(3)
    

In [15]:
alpha_seattle = {}
for i in [0,1,2,4,5,7,8,9]:
    alpha = krippendorff_alpha(seattle_labels,i, seattle_streets, 2285)
    alpha_seattle[label_types[i]] = alpha

In [17]:
#30 ft
alpha_seattle

{'CurbRamp': 0.827,
 'NoSidewalk': 0.683,
 'Problem': 0.44,
 'SurfaceProblem': 0.464,
 'Obstacle': 0.418,
 'NoCurbRamp': 0.875,
 'Crosswalk': 0.853,
 'Signal': 0.454}

In [18]:
for i in [0,1,2,4,5,7,8,9]:
    labels= map_clusters(seattle_labels,i)
    map = seattle_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"], popup=["street_edge_id"])
    map.save("html/seattle_"+label_types[i]+"_230608.html")


In [19]:
#take out label type = surfaceproblem and obstacle
seattle_problems= seattle_labels[(seattle_labels['label_type'] == 'SurfaceProblem') | (seattle_labels['label_type'] == 'Obstacle') ]
#take out severity ==1
seattle_problems = seattle_problems[seattle_problems['severity'] != 1]
seattle_problems

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,3,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,t,3.0,f,pole,None,POINT (-122.32071 47.66104)
1,4,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,t,2.0,f,pole,None,POINT (-122.32070 47.66103)
2,5,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,t,2.0,f,bumpy,None,POINT (-122.32071 47.66106)
4,9,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,t,3.0,f,sign,temporary due to construction,POINT (-122.32066 47.66110)
18,30,Devon Snyder,30,483,University District,Obstacle,47.661419,-122.320488,iI69DtsYap_Fjiz-RZ4i1A,f,2.0,f,pole,None,POINT (-122.32049 47.66142)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,2485,mikey,2485,810,Windermere,SurfaceProblem,47.672047,-122.267578,hVSZIyYVmFvFvQ6Rizfo8g,None,2.0,f,None,pine needles,POINT (-122.26758 47.67205)
1451,2486,mikey,2486,810,Windermere,SurfaceProblem,47.672070,-122.267845,09WtcNVwH9KSZ8B4Hqm3Bg,None,2.0,f,None,pine needles,POINT (-122.26785 47.67207)
1452,2487,mikey,2487,810,Windermere,Obstacle,47.672207,-122.267700,09WtcNVwH9KSZ8B4Hqm3Bg,None,4.0,f,pole,None,POINT (-122.26770 47.67221)
1469,2534,Devon Snyder,2534,479,University District,SurfaceProblem,47.659721,-122.319923,SRZ-Py452sb8rEJfkHzHWw,None,2.0,f,"cracks,uneven/slanted",None,POINT (-122.31992 47.65972)


In [20]:
alpha_seattle_problems = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(seattle_problems,i, seattle_streets, 3435)
    alpha_seattle_problems[label_types[i]] = alpha

In [21]:
alpha_seattle_problems

{'Problem': 0.426, 'SurfaceProblem': 0.407, 'Obstacle': 0.472}

In [18]:
for i in [2,4,5]:
    labels= map_clusters(seattle_problems,i)
    map = seattle_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/seattle_wo_sev1_"+label_types[i]+"_230608.html")